In [1]:
import time
notebookstart= time.time()

In [2]:
import os
#from typing import Tuple

import pandas as pd
#import numpy as np

In [3]:
DIR_SUBM = os.path.join(os.getcwd(), 'subm')
DIR_DATA = os.path.join(os.getcwd(), 'data')
DIR_DATA_TRAIN = os.path.join(DIR_DATA, 'train')
DIR_DATA_TEST  = os.path.join(DIR_DATA, 'test')

In [4]:
!pip install catboost

In [5]:
from catboost import CatBoostRegressor
from catboost import Pool, cv

In [6]:
train_df = pd.read_csv(os.path.join(DIR_DATA, 'train_upd.csv'))
test_df  = pd.read_csv(os.path.join(DIR_DATA, 'test_upd.csv'))
train_df.shape, test_df.shape

((530, 10), (517, 9))

In [7]:
train_df.head()

,image_name,distance,x_min,y_min,x_max,y_max,conf,class,h,w
0,img_1596.jpg,4.88,1664.946045,1180.270630,2279.794434,1715.034180,0.765476,2.0,534.763550,614.848389
1,img_1600.jpg,1.54,518.410889,1144.958374,1430.319336,1757.429810,0.879383,2.0,612.471436,911.908447
2,img_1601.jpg,3.68,2443.314697,1339.932617,3042.800781,1779.703003,0.869439,2.0,439.770386,599.486084
3,img_1603.jpg,2.22,2609.617920,1269.794922,3321.687988,1847.752930,0.911404,2.0,577.958008,712.070068
4,img_1605.jpg,3.73,2485.785889,1233.717041,3120.986328,1677.980591,0.891404,2.0,444.263550,635.200439


In [8]:
train_df['class'].value_counts()

2.0    526
Name: class, dtype: int64

In [9]:
%%time
params = {"iterations": 3500,
          "loss_function": 'RMSE',
          #"loss_function": 'R2',
         }

train = Pool(data = train_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf', 'h', 'w']],
             label = train_df[['distance']],
             #cat_features=cat_features
            )

scores = cv(train, params,
            fold_count = 3,
            verbose = False,
           )

Training on fold [0/3]

bestTest = 0.9742899502
bestIteration = 245

Training on fold [1/3]

bestTest = 0.7885688008
bestIteration = 1111

Training on fold [2/3]

bestTest = 0.8250481807
bestIteration = 1100

CPU times: total: 1min 45s
Wall time: 20.2 s


In [10]:
niter = scores['test-RMSE-mean'].argmin() + 13
scores['test-RMSE-mean'].min(), scores['test-RMSE-mean'].argmin(), niter

(0.8689944606627914, 608, 621)

In [11]:
%%time
#model_cb = CatBoostRegressor(iterations = niter, verbose = 100)
model_cb = CatBoostRegressor(verbose = 100)

# Fit model
model_cb.fit(train_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf', 'h', 'w']], train_df[['distance']].values)

Learning rate set to 0.037035
0:	learn: 1.7177959	total: 1.81ms	remaining: 1.81s
100:	learn: 0.6993281	total: 144ms	remaining: 1.28s
200:	learn: 0.5640105	total: 266ms	remaining: 1.05s
300:	learn: 0.4773828	total: 375ms	remaining: 871ms
400:	learn: 0.4139589	total: 496ms	remaining: 741ms
500:	learn: 0.3692204	total: 608ms	remaining: 605ms
600:	learn: 0.3309356	total: 728ms	remaining: 483ms
700:	learn: 0.3017665	total: 846ms	remaining: 361ms
800:	learn: 0.2757274	total: 965ms	remaining: 240ms
900:	learn: 0.2544207	total: 1.11s	remaining: 122ms
999:	learn: 0.2365099	total: 1.23s	remaining: 0us
CPU times: total: 7.69 s
Wall time: 1.58 s


In [12]:
preds = model_cb.predict(test_df[['x_min', 'y_min', 'x_max', 'y_max', 'conf', 'h', 'w']])
test_df['distance'] = preds

sample_solution_df = test_df[['image_name', 'distance']]

In [13]:
test_img_names = set(os.listdir(DIR_DATA_TEST))

In [14]:
lost_test_items = []

for file_name in test_img_names - set(sample_solution_df['image_name'].values):
    lost_test_items.append([file_name, 0])
    
lost_test_items_df = pd.DataFrame(lost_test_items, columns=['image_name', 'distance'])
sample_solution_df = pd.concat([sample_solution_df, lost_test_items_df])

sample_solution_df.to_csv(os.path.join(DIR_SUBM, '6_baseline_hw_wnulls.csv'), sep=';', index=False)